# Calculating and rendering range retention in Pyhton

2/12/2019

Raphael Vannson

## Abstract
This post explains what range retention is, provides a strategy to calculate it and a link to a Python / Jupyter notebook providing a complete hands on example.

## What is `Range Retention`?

### Simple question... simple answer?

Retention is an observational metric used by organizations to evaluate how much of their members come back after a  first contact. For example: brick and mortar shop owners want to know if customers come back after their first purchase, web platforms want to know if users visit come back after they have created an account, etc...

So what's the big deal I hear you ask? Sounds like retention can easily be defined as a single number given by this formula:

$$
\begin{equation}
R = \frac{Number\ of\ unique\ users\ who\ interacted\ more\ than\ once \ with\ the\ organization}{Number\ of\ unique\ users\ who\ interacted\ with\ the\ organization}
\end{equation}
$$

### Not so fast...
The problem with this definition of retention is that it is an average over all users since the organization first day of business. It does not say much about the organization's current ability to get its members to come back nor does it measure the (potential) effect of actions taken by the organization to improve retention. 

### `Range Retention` to the rescue!

One (popular) way to address this is to get a tad more refined by calculating multiple (relevant) retention rates - `Range Retention` does just that. It provides a granular view into a long period of interest by breaking it down into small windows. This makes it possible to track how users of each window behaved throughout the rest of the long period of interest. Sounds cryptic? Nah... Here is a simple example: Say our long period of interest is 2018 and that we break it down by month, then our range retention is going to be a bunch of retention rates focusing on a specific user cohort at a specific month: 

 * Users who made first contact in January: what proportion came back in February, March, ..., December?
 * Users who made first contact in February: what proportion came back in March, April, ..., December?
 * ...
 * Users who made first contact in November: what proportion came back in December?


So, `Range Retention` consists in:

 * Defining a period of interest (ex: the last 12 months) 
 * Breaking down the period of interest into small chunck (ex: month, week)
 * Identifying the cohort of users or each chunk (all the users who made first contact during the chunk)
 * For each cohort calculate the percentage of users in the cohort who came back in the subsequent chunks
 
 
`Range Retention` will present itself as a triangular table with one row per user cohort and one column per chunk.

## How to calculate `Range Retention`?

In this section we provide a data manipulation approach to calculate `Range Retention`.

### The basics
Ensure your data is loaded in a dataframe of your favorite flavor and that the column types are what they need to be. In addition make sure there is one column capturing the "chunk ID" (ex: month or week number). 

### Define the cohorts

You will want to create a dataframe wich maps 

In [4]:
import pandas as pd
from datetime import datetime

# Load data from CSV

In [14]:
orders_df = pd.read_csv('./orders.csv')
orders_df.dtypes

orderid          int64
orderdate       object
customerid       int64
netamount      float64
tax            float64
totalamount    float64
dtype: object

In [15]:
orders_df.head()

,orderid,orderdate,customerid,netamount,tax,totalamount
0,1,2004-01-27,7888,313.24,25.84,339.08
1,2,2004-01-01,4858,54.90,4.53,59.43
2,3,2004-01-17,15399,160.10,13.21,173.31
3,4,2004-01-28,17019,106.67,8.80,115.47
4,5,2004-01-09,14771,256.00,21.12,277.12


# Change the `orderdate` column type from String to a Datetime

In [16]:
# used to set column type
orders_df['orderdate'] = orders_df['orderdate'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
orders_df.dtypes

orderid                 int64
orderdate      datetime64[ns]
customerid              int64
netamount             float64
tax                   float64
totalamount           float64
dtype: object

# Range retention

In [5]:
# Define cohorts by the month of the first order

# Take the first order date per customer
cohorts_df = orders_df \
.groupby('customerid')['orderdate'] \
.agg('min') \
.pipe(pd.DataFrame)


# Customerid is in the index
# Create a column with the customerid
cohorts_df = cohorts_df.reset_index()


# Extract the month from the order date
def get_month(date):
    return(date.month)

cohorts_df['join_month'] = cohorts_df['orderdate'].apply(lambda x: get_month(x))


# Drop the date column
cohorts_df = cohorts_df.drop('orderdate', axis = 1)

# Each customer Id is associated with the month
# they first purchased (join_month) - that is also
# the cohort ID
cohorts_df.head()

,customerid,join_month
0,2,11
1,3,3
2,6,10
3,7,7
4,11,2


In [6]:
# Count cohort sizes

cohort_sizes_df = cohorts_df \
.groupby('join_month')['customerid'] \
.agg('count') \
.rename('cohort_size') \
.pipe(pd.DataFrame) \
.reset_index()

cohort_sizes_df

,join_month,cohort_size
0,1,979
1,2,930
2,3,882
3,4,835
4,5,792
5,6,763
6,7,742
7,8,689
8,9,651
9,10,632


In [7]:
# Cohort activity for each month
# One row per customer who came back at least once
# 'activity_index' months after activation

# Get the activity month
df = pd.merge(orders_df, cohorts_df,
         how = 'inner',
         left_on = 'customerid',
         right_on = 'customerid')


# Add the activity month and index
df['activity_month'] = df['orderdate'].apply(lambda x: get_month(x))
df['activity_index'] = df['activity_month'] - df['join_month']


# Remove activity for the first month
df = df[df.activity_index > 0]


# Make sure there is a max of one row per customer per month
# (customers who came back at least once in a month)
df = df[['customerid', 'join_month', 'activity_index']].drop_duplicates()

df.head()

,customerid,join_month,activity_index
2,4858,1,11
4,15399,1,2
6,17019,1,2
7,17019,1,3
8,17019,1,7


In [8]:
# Count the number of returning customers
# per cohort, per month

activity_size_df = df \
.groupby(['join_month', 'activity_index'])['customerid'] \
.agg('count') \
.rename('activity_size') \
.pipe(pd.DataFrame) \
.reset_index()

activity_size_df.head()

,join_month,activity_index,activity_size
0,1,1,43
1,1,2,46
2,1,3,54
3,1,4,43
4,1,5,47


In [9]:
# Calculate the retention rate
# per month after activation for each cohort

retention_df = pd.merge(cohort_sizes_df, activity_size_df,
        how = 'inner',
        left_on = 'join_month',
        right_on = 'join_month')


retention_df['retention_rate'] = round(retention_df['activity_size'] / retention_df['cohort_size'] * 100, 2)
retention_df.head()

,join_month,cohort_size,activity_index,activity_size,retention_rate
0,1,979,1,43,4.39
1,1,979,2,46,4.70
2,1,979,3,54,5.52
3,1,979,4,43,4.39
4,1,979,5,47,4.80


In [11]:
retention_tbl = retention_df \
.pivot_table(values = 'retention_rate',
             index = ['join_month', 'cohort_size'], 
             columns = 'activity_index')

retention_tbl

,activity_index,1,2,3,4,5,6,7,8,9,10,11
join_month,cohort_size,,,,,,,,,,,
1,979,4.39,4.70,5.52,4.39,4.80,4.60,3.68,5.21,4.39,5.11,5.01
2,930,4.52,5.27,6.13,5.38,3.66,4.73,4.52,4.19,4.95,3.98,NaN
3,882,4.31,5.33,2.95,4.20,4.42,5.33,3.29,5.10,6.35,NaN,NaN
4,835,4.31,5.39,4.31,6.35,4.79,4.91,4.91,5.27,NaN,NaN,NaN
5,792,5.30,5.56,3.41,4.80,4.67,4.80,4.92,NaN,NaN,NaN,NaN
6,763,5.64,4.72,4.72,6.42,4.06,5.50,NaN,NaN,NaN,NaN,NaN
7,742,6.47,4.18,5.66,4.58,4.99,NaN,NaN,NaN,NaN,NaN,NaN
8,689,4.35,4.64,5.22,5.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,651,5.53,5.22,4.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


"Plot" the table to make it easier to find the high / low retention cells and visally detect wether there is a pattern appears.

In [13]:
# NaNs are replaced by 0s style.background_gradient() does not like columns with NaNs starting with python 3.7...
retention_tbl.fillna(0).style.background_gradient(cmap = 'Greens')

# See https://matplotlib.org/tutorials/colors/colormaps.html
# for valid cmaps. Examples:
#retention_tbl.style.background_gradient(cmap = 'Blues')
#retention_tbl.style.background_gradient(cmap = 'Wistia')

,activity_index,1,2,3,4,5,6,7,8,9,10,11
join_month,cohort_size,,,,,,,,,,,
1,979,4.39,4.7,5.52,4.39,4.8,4.6,3.68,5.21,4.39,5.11,5.01
2,930,4.52,5.27,6.13,5.38,3.66,4.73,4.52,4.19,4.95,3.98,0
3,882,4.31,5.33,2.95,4.2,4.42,5.33,3.29,5.1,6.35,0,0
4,835,4.31,5.39,4.31,6.35,4.79,4.91,4.91,5.27,0,0,0
5,792,5.3,5.56,3.41,4.8,4.67,4.8,4.92,0,0,0,0
6,763,5.64,4.72,4.72,6.42,4.06,5.5,0,0,0,0,0
7,742,6.47,4.18,5.66,4.58,4.99,0,0,0,0,0,0
8,689,4.35,4.64,5.22,5.22,0,0,0,0,0,0,0
9,651,5.53,5.22,4.3,0,0,0,0,0,0,0,0
